In [1]:
from common_imports import *
show_home_button()
from db_connection import get_engine
engine = get_engine()

▶ Python executable: c:\Users\StanvanBon\Miniconda3\envs\energymonitor_env\python.exe
▶ dotenv module: c:\Users\StanvanBon\Miniconda3\envs\energymonitor_env\Lib\site-packages\dotenv\__init__.py


Output()

In [2]:
try:
    import xlsxwriter
except ImportError:
    print("Module 'xlsxwriter' ontbreekt.")

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

RATE_LIMIT = 3
RATE_PERIOD = 300  # 5 minuten

rate_lock = threading.Lock()
rate_calls = deque()

def check_rate_limit() -> Tuple[bool, int]:
    with rate_lock:
        now = time.time()
        while rate_calls and (now - rate_calls[0] > RATE_PERIOD):
            rate_calls.popleft()
        if len(rate_calls) >= RATE_LIMIT:
            wait_time = int(RATE_PERIOD - (now - rate_calls[0]))
            return False, wait_time
        rate_calls.append(now)
        return True, 0

In [3]:
class TTLCache:
    def __init__(self, ttl: int = 300):
        self.cache = {}
        self.ttl = ttl
        self.lock = threading.Lock()

    def get(self, key):
        with self.lock:
            entry = self.cache.get(key)
            if entry is None:
                return None
            value, expires_at = entry
            if time.time() > expires_at:
                del self.cache[key]
                return None
            return value

    def set(self, key, value):
        with self.lock:
            expires_at = time.time() + self.ttl
            self.cache[key] = (value, expires_at)

    def clear(self):
        with self.lock:
            self.cache.clear()

min_max_cache = TTLCache(ttl=300)
full_data_cache = TTLCache(ttl=300)

In [ ]:
from mappings import get_typeids, validate_unique_ids
validate_unique_ids()

LDN_TYPEIDS = get_typeids("Hoofdmeting elektriciteit LDN")
ODN_TYPEIDS = get_typeids("Hoofdmeting elektriciteit ODN")

def fetch_typeids_for_aansluiting(aansluitnummer: str) -> List[int]:
    query = """
    SELECT DISTINCT r.TypeId
    FROM TBL_Register r
    JOIN TBL_ConnectionPoint cp ON cp.ID = r.ConnectionPointId
    WHERE cp.EAN_ConnectionPoint = ?
    """
    try:
        with engine.connect() as conn:
            df_temp = pd.read_sql_query(query, conn, params=(aansluitnummer,))
        if df_temp.empty:
            return []
        return df_temp['TypeId'].tolist()
    except Exception as e:
        logger.error(f"Error fetching TypeIDs for {aansluitnummer}: {e}")
        return []

def fetch_min_max_period(aansluitnummer: str,
                         start_date: datetime,
                         end_date: datetime
                         ) -> Tuple[Optional[datetime], Optional[datetime]]:
    cache_key = (aansluitnummer, start_date, end_date, 'minmax')
    cached = min_max_cache.get(cache_key)
    if cached is not None:
        logger.info(f"[CACHE HIT] Min/Max voor {aansluitnummer} uit cache.")
        return cached

    sp_query = """
        EXEC [dbo].[usp_GetMinMaxPeriod_OnlyLDNODN]
             @EAN_ConnectionPoint = ?,
             @StartDate = ?,
             @EndDate = ?
    """
    try:
        with engine.connect() as conn:
            df_temp = pd.read_sql_query(sp_query, conn,
                                        params=(aansluitnummer, start_date, end_date))
        if df_temp.empty or pd.isnull(df_temp['MinPeriod'].iloc[0]):
            result = (None, None)
        else:
            result = (df_temp['MinPeriod'].iloc[0], df_temp['MaxPeriod'].iloc[0])
        min_max_cache.set(cache_key, result)
        logger.info(f"Cached Min/Max periode voor {aansluitnummer}.")
        return result
    except Exception as e:
        logger.error(f"Error fetching min/max voor {aansluitnummer}: {e}")
        return (None, None)

def fetch_full_data(aansluitnummer: str,
                    start_date: datetime,
                    end_date: datetime
                    ) -> Optional[pd.DataFrame]:
    cache_key = (aansluitnummer, start_date, end_date, 'pivot')
    cached = full_data_cache.get(cache_key)
    if cached is not None:
        logger.info(f"[CACHE HIT] Pivot-data voor {aansluitnummer} uit cache.")
        return cached

    sp_query = """
        EXEC [dbo].[usp_GetConnectionDataFull_OnlyLDNODN]
             @EAN_ConnectionPoint = ?,
             @StartDate = ?,
             @EndDate = ?
    """
    try:
        with engine.connect() as conn:
            df = pd.read_sql_query(sp_query, conn,
                                   params=(aansluitnummer, start_date, end_date),
                                   parse_dates=['utcperiod'])
        if df.empty:
            result = None
        else:
            result = df
        full_data_cache.set(cache_key, result)
        logger.info(f"Cached data voor {aansluitnummer}.")
        return result
    except Exception as e:
        logger.error(f"Error fetching pivot-data voor {aansluitnummer}: {e}")
        return None

FREQ_TO_SECONDS = {
    '5min': 300,
    '15min': 900,
    'h': 3600,
    'D': 86400,
    'W': 604800,
    'M': 2592000,
    'Y': 31536000
}

def distribute_consumption_across_intervals(df: pd.DataFrame, freq: str) -> pd.DataFrame:
    if df.empty:
        return df
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df.index = pd.to_datetime(df.index)

    numeric_df = df.select_dtypes(include=[np.number])
    non_numeric_df = df.select_dtypes(exclude=[np.number])
    numeric_df = numeric_df.groupby(numeric_df.index).sum()
    cumsum_numeric = numeric_df.cumsum()
    new_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
    cumsum_numeric = cumsum_numeric.reindex(new_index).interpolate(method='linear')
    distributed_numeric = cumsum_numeric.diff()
    if not distributed_numeric.empty:
        distributed_numeric.iloc[0] = cumsum_numeric.iloc[0]
    distributed_numeric = distributed_numeric.fillna(0)

    if not non_numeric_df.empty:
        non_numeric_df = non_numeric_df.groupby(non_numeric_df.index).first()
        non_numeric_df = non_numeric_df.reindex(new_index, method='ffill')
        result = pd.concat([non_numeric_df, distributed_numeric], axis=1)
    else:
        result = distributed_numeric

    return result

def combine_to_new_outputformat(df: pd.DataFrame) -> pd.DataFrame:
    columns_lower = [c.lower() for c in df.columns]
    if "utcperiod" in columns_lower:
        date_col = df.columns[columns_lower.index("utcperiod")]
    else:
        date_col = df.columns[0]

    ldn_cols = []
    odn_cols = []

    for col in df.columns:
        if col.lower() == date_col.lower():
            continue
        match = re.search(r"\((\d+)\)", str(col))
        if match:
            rid = int(match.group(1))
            if rid in LDN_TYPEIDS:
                ldn_cols.append(col)
            elif rid in ODN_TYPEIDS:
                odn_cols.append(col)

    if not ldn_cols and not odn_cols:
        for col in df.columns:
            if col.lower() == date_col.lower():
                continue
            low = str(col).lower()
            if "ldn" in low:
                ldn_cols.append(col)
            elif "odn" in low:
                odn_cols.append(col)

    df_out = pd.DataFrame()
    df_out["Datum"] = df[date_col]
    df_out["Afname"] = df[ldn_cols].sum(axis=1, skipna=False) if ldn_cols else np.nan
    df_out["Invoeding"] = df[odn_cols].sum(axis=1, skipna=False) if odn_cols else np.nan

    return df_out

In [5]:
def build_dataset(aansluitnummer: str,
                  start_date: datetime,
                  end_date: datetime,
                  freq_val: str
                  ) -> Optional[pd.DataFrame]:
    from_db_min, from_db_max = fetch_min_max_period(aansluitnummer, start_date, end_date)
    if not from_db_min or not from_db_max:
        logger.info(f"[DEBUG] Geen data voor {aansluitnummer}")
        return None

    df = fetch_full_data(aansluitnummer, start_date, end_date)
    if df is None or df.empty:
        logger.info(f"[DEBUG] Geen pivot-data voor {aansluitnummer}")
        return None

    df_f = df[(df["utcperiod"] >= start_date) & (df["utcperiod"] <= end_date)].copy()
    if df_f.empty:
        return None

    numeric_cols = [c for c in df_f.columns if c.lower() != "utcperiod"]
    for c in numeric_cols:
        df_f[c] = pd.to_numeric(df_f[c], errors="coerce")

    df_f.set_index("utcperiod", inplace=True)
    df_dist = distribute_consumption_across_intervals(df_f, freq_val)
    df_dist = df_dist.reset_index().rename(columns={"index": "utcperiod"})

    final_df = combine_to_new_outputformat(df_dist)
    if final_df.empty:
        logger.info(f"[DEBUG] Lege dataset na combine voor {aansluitnummer}")
        return None

    return final_df

def detect_global_frequency(aansluit_list: List[str],
                            start_date: datetime,
                            end_date: datetime) -> str:
    all_diffs = []
    for ansl in aansluit_list:
        df = fetch_full_data(ansl, start_date, end_date)
        if df is None or df.empty:
            continue
        df_f = df[(df["utcperiod"] >= start_date) & (df["utcperiod"] <= end_date)].copy()
        if df_f.empty:
            continue
        df_f.sort_values("utcperiod", inplace=True)
        vals = df_f["utcperiod"].unique()
        if len(vals) > 1:
            diffs = (vals[1:] - vals[:-1]).astype('timedelta64[s]').astype(int)
            all_diffs.extend(diffs.tolist())

    if not all_diffs:
        return "h"

    s = pd.Series(all_diffs)
    mode_diff = s.mode().iloc[0]
    for freq_str, freq_secs in FREQ_TO_SECONDS.items():
        if abs(freq_secs - mode_diff) < 2:
            return freq_str
    return "h"

def build_multiean_data(aansluit_list: List[str],
                        start_date: datetime,
                        end_date: datetime,
                        freq_val: str,
                        progress_callback: Optional[Callable[[int, str], None]] = None
                        ) -> Optional[pd.DataFrame]:
    """
    Bouwt een gecombineerde DataFrame voor meerdere EANs.
    Als er voor een EAN geen data is, wordt de boodschap  in de eerste rij weergegeven.
    """
    if not aansluit_list:
        return None
    if freq_val.lower() == 'auto':
        freq_val = detect_global_frequency(aansluit_list, start_date, end_date)
        logger.info(f"[DEBUG] freq = {freq_val}")

    combined_df = None
    total = len(aansluit_list)
    for i, ansl in enumerate(aansluit_list):
        if progress_callback:
            pct = 20 + int((i / total) * 50)
            progress_callback(pct, f"Data voor {ansl} ({i+1}/{total})")

        df_ansl = build_dataset(ansl, start_date, end_date, freq_val)

        if df_ansl is None or df_ansl.empty:
            typeids = fetch_typeids_for_aansluiting(ansl)
            message = "EAN niet aanwezig" if not typeids else "Geen data beschikbaar"

            try:
                dummy_index = pd.date_range(start=start_date, end=end_date, freq=freq_val)
            except Exception:
                dummy_index = pd.date_range(start=start_date, end=end_date, freq="h")

            afname_vals = [message] + [""] * (len(dummy_index) - 1)
            invoeding_vals = [message] + [""] * (len(dummy_index) - 1)

            df_ansl = pd.DataFrame({
                "Datum": dummy_index,
                "Afname": afname_vals,
                "Invoeding": invoeding_vals
            })
        else:
            df_ansl.sort_values("Datum", inplace=True)

        df_ansl.set_index("Datum", inplace=True)
        df_ansl.columns = pd.MultiIndex.from_tuples(
            [(ansl, col) for col in df_ansl.columns],
            names=[None, None]
        )

        if combined_df is None:
            combined_df = df_ansl
        else:
            combined_df = pd.concat([combined_df, df_ansl], axis=1)

    if combined_df is None or combined_df.empty:
        return None

    if progress_callback:
        progress_callback(75, "Combineren...")

    combined_df.reset_index(inplace=True)
    logger.info(f"[DEBUG] build_multiean_data: shape={combined_df.shape}")
    return combined_df

# CSV-EXPORTFUNCTIE: per EAN 2 kolommen (Afname/Invoeding)
def export_dataset_to_csv(df: pd.DataFrame, filename: str) -> bool:
    if df is None or df.empty:
        logger.warning("[CSV] DataFrame is leeg. Export wordt overgeslagen.")
        return False

    logger.info(f"[CSV] Export naar kolom-CSV: {filename}  (shape={df.shape})")

    def split_decimal(num) -> (str, str):
        if pd.isna(num):
            return "", ""
        if not isinstance(num, (int, float, np.number)):
            return str(num), ""
        txt = f"{num:.2f}"
        parts = txt.split('.')
        int_part, frac_part = parts[0], parts[1]
        if frac_part == "00":
            frac_part = "0"
        return int_part, frac_part

    columns_list = list(df.columns)
    col_datum = columns_list[0]
    ean_cols = columns_list[1:]

    ean_order = []
    for c in ean_cols:
        ean_name = c[0]
        if ean_name not in ean_order:
            ean_order.append(ean_name)

    header_parts = ["Datum tijd"]
    for ean in ean_order:
        header_parts += [f"{ean}_Afname", f"{ean}_Invoeding"]

    data_lines = []
    for row_i in range(len(df)):
        row_vals = []
        val_datum = df.iloc[row_i][col_datum]
        try:
            dt = val_datum if isinstance(val_datum, (datetime, pd.Timestamp)) else pd.to_datetime(val_datum)
            formatted_date = dt.strftime("%-d-%-m-%Y %H:%M") if os.name != 'nt' else dt.strftime("%#d-%#m-%Y %H:%M")
        except:
            formatted_date = str(val_datum)
        row_vals.append(formatted_date)

        for ean in ean_order:
            val_afn = df.iloc[row_i].get((ean, "Afname"), np.nan)
            val_inv = df.iloc[row_i].get((ean, "Invoeding"), np.nan)
            afn_int, afn_dec = split_decimal(val_afn)
            inv_int, inv_dec = split_decimal(val_inv)
            combined_afn = afn_int if afn_dec in ("", "0") else f"{afn_int},{afn_dec}"
            combined_inv = inv_int if inv_dec in ("", "0") else f"{inv_int},{inv_dec}"
            row_vals += [combined_afn, combined_inv]

        data_lines.append("\t".join(row_vals))

    try:
        with open(filename, "w", encoding="utf-8", newline="") as f:
            f.write("\t".join(header_parts) + "\n")
            for line in data_lines:
                f.write(line + "\n")
        logger.info(f"[CSV] Kolom-CSV opgeslagen: {filename}")
        return True
    except Exception as e:
        logger.error("[CSV] Fout bij CSV-export:", exc_info=True)
        return False

# EXCEL-EXPORTFUNCTIE: per EAN 2 kolommen (Afname/Invoeding)
def export_dataset_to_excel(df: pd.DataFrame, filename: str) -> bool:
    if df is None or df.empty:
        logger.warning("[XLS] DataFrame is leeg. Excel-export wordt overgeslagen.")
        return False

    logger.info(f"[XLS] Export => {filename}, shape={df.shape}")

    columns_list = list(df.columns)
    col_datum = columns_list[0]
    ean_cols = columns_list[1:]

    ean_order = []
    for c in ean_cols:
        if c[0] not in ean_order:
            ean_order.append(c[0])

    line1_parts = ["EAN"]
    for ean_name in ean_order:
        line1_parts += [ean_name, ean_name]

    line2_parts = ["Datum tijd"]
    for _ in ean_order:
        line2_parts += ["Afname", "Invoeding"]

    def split_decimal(num) -> (str, str):
        if pd.isna(num):
            return "", ""
        if not isinstance(num, (int, float, np.number)):
            return str(num), ""
        txt = f"{num:.2f}"
        parts = txt.split('.')
        int_part, frac_part = parts[0], parts[1]
        if frac_part == "00":
            frac_part = "0"
        return int_part, frac_part

    data_matrix = []
    for i in range(len(df)):
        row_vals = []
        val_datum = df.iloc[i][col_datum]
        try:
            dt = val_datum if isinstance(val_datum, (datetime, pd.Timestamp)) else pd.to_datetime(val_datum)
            formatted_date = dt.strftime("%-d-%-m-%Y %H:%M") if os.name != 'nt' else dt.strftime("%#d-%#m-%Y %H:%M")
        except:
            formatted_date = str(val_datum)
        row_vals.append(formatted_date)

        for ean in ean_order:
            val_afname = df.iloc[i].get((ean, "Afname"), np.nan)
            val_invoeding = df.iloc[i].get((ean, "Invoeding"), np.nan)
            main_val = val_afname if not (pd.isna(val_afname) or val_afname == 0) else val_invoeding
            afn_str, inv_str = split_decimal(main_val)
            row_vals += [afn_str, inv_str]

        data_matrix.append(row_vals)

    try:
        with xlsxwriter.Workbook(filename) as workbook:
            ws = workbook.add_worksheet("Dataset")
            header_format = workbook.add_format({
                'bold': True, 'align': 'center', 'valign': 'vcenter',
                'border': 1
            })
            data_format = workbook.add_format({
                'align': 'center', 'valign': 'vcenter',
                'border': 1
            })

            for col_idx, val in enumerate(line1_parts):
                ws.write(0, col_idx, val, header_format)
            for col_idx, val in enumerate(line2_parts):
                ws.write(1, col_idx, val, header_format)

            for row_idx, row_vals in enumerate(data_matrix, start=2):
                for col_idx, val in enumerate(row_vals):
                    ws.write(row_idx, col_idx, val, data_format)

            ws.set_column(0, 0, 18)
            for c in range(1, len(line1_parts)):
                ws.set_column(c, c, 12)

        logger.info("[XLS] OK.")
        return True
    except Exception as e:
        logger.error("[XLS] Fout bij Excel-export:", exc_info=True)
        traceback.print_exc()
        return False

In [ ]:
try:
    prefilled_eans
except NameError:
    prefilled_eans = []

# EAN-textarea
ean_textarea = widgets.Textarea(
    value="\n".join(prefilled_eans),
    placeholder='Plak hier de EANs, één per regel',
    description='EANs:',
    layout=widgets.Layout(width='600px', height='100px')
)

# Datum-pickers en frequentie-dropdown
start_date_picker = widgets.DatePicker(
    description='Start:',
    value=date.today() - relativedelta(years=2),
    layout=widgets.Layout(width='250px', height='2.5rem')
)
end_date_picker = widgets.DatePicker(
    description='Eind:',
    value=date.today(),
    layout=widgets.Layout(width='250px', height='2.5rem')
)
freq_selector = widgets.Dropdown(
    options=[
        ('Automatisch','auto'),
        ('5 min','5min'),
        ('15 min','15min'),
        ('Uur','h'),
        ('Dag','D'),
        ('Week','W'),
        ('Maand','M'),
        ('Jaar','Y'),
    ],
    value='auto',
    description='Freq:',
    layout=widgets.Layout(width='180px', height='2.5rem')
)

# Filter- & actie-knoppen (zelfde hoogte)
btn_load_filters   = widgets.Button(description='Zoeken',      button_style='info',    icon='search',    layout=widgets.Layout(height='2.5rem'))
btn_reset_filters  = widgets.Button(description='Reset',       button_style='warning', icon='refresh',   layout=widgets.Layout(height='2.5rem'))
btn_build_dataset  = widgets.Button(description='Haal data op', button_style='success', icon='database',  layout=widgets.Layout(height='2.5rem'))
btn_view_dataset   = widgets.Button(description='Tabelweergave', button_style='primary', icon='table',    disabled=True, layout=widgets.Layout(height='2.5rem'))
btn_download_csv   = widgets.Button(description='Download CSV',  button_style='primary', icon='download', disabled=True, layout=widgets.Layout(height='2.5rem'))
btn_download_excel = widgets.Button(description='Download XLS',  button_style='primary', icon='file-excel-o', disabled=True, layout=widgets.Layout(height='2.5rem'))

# Progress & output areas
progress_bar       = widgets.IntProgress(min=0, max=100, description='Voortgang:')
status_label       = widgets.Label(value="")
progress_container = widgets.HBox([progress_bar, status_label])
progress_container.layout.visibility = 'hidden'

output_area        = widgets.Output()
data_table_output  = widgets.Output(layout={
    'border':'1px solid #ccc',
    'display':'none',
    'overflow_x':'auto',
    'overflow_y':'auto',
    'max_height':'400px',
    'width':'100%'
})

# HBox-rijen met flex-wrap **en baseline-alignment**
row_dates = widgets.HBox(
    [start_date_picker, end_date_picker, freq_selector],
    layout=widgets.Layout(
        display='flex',
        flex_flow='row wrap',
        align_items='baseline',   # baseline uitlijning
        gap='10px'
    )
)
row_filter_buttons = widgets.HBox(
    [btn_load_filters, btn_reset_filters, btn_build_dataset,
     btn_view_dataset, btn_download_csv, btn_download_excel],
    layout=widgets.Layout(
        display='flex',
        flex_flow='row wrap',
        align_items='baseline',   # baseline uitlijning
        gap='5px'
    )
)

# Filters container
filters_container = widgets.VBox(
    [
        widgets.HTML("<h3>Filters</h3>"),
        ean_textarea,
        row_dates,
        row_filter_buttons,
        progress_container,
        output_area,
        data_table_output
    ],
    layout=widgets.Layout(
        display='flex',
        flex_flow='column wrap',
        gap='10px',
        border='1px solid #ccc',
        padding='5px'
    )
)

# Toggle knop om filters te verbergen/tonen
toggle_filters_button = widgets.Button(
    description='Verberg',
    icon='chevron-up',
    button_style='info',
    layout=widgets.Layout(margin='0 0 10px 0', height='2.5rem')
)

def toggle_filters_display(_):
    if filters_container.layout.display == 'none':
        filters_container.layout.display = 'flex'
        toggle_filters_button.description = "Verberg"
        toggle_filters_button.icon = "chevron-up"
    else:
        filters_container.layout.display = 'none'
        toggle_filters_button.description = "Toon filters"
        toggle_filters_button.icon = "chevron-down"

toggle_filters_button.on_click(toggle_filters_display)

# Final UI
final_ui = widgets.VBox(
    [toggle_filters_button, filters_container],
    layout=widgets.Layout(width='100%')
)

display(final_ui)


def update_progress(value, msg="", error=False):
    progress_container.layout.visibility = 'visible'
    progress_bar.value = value
    status_label.value = msg
    if error:
        progress_bar.bar_style = 'danger'
    elif value >= 100:
        progress_bar.bar_style = 'success'
    else:
        progress_bar.bar_style = 'info'

def finish_progress():
    time.sleep(0.5)
    progress_container.layout.visibility = 'hidden'
    progress_bar.value = 0
    status_label.value = ""
    progress_bar.bar_style = 'info'

combined_data = None

def get_aansluit_list():
    lines = ean_textarea.value.splitlines()
    return [line.strip() for line in lines if line.strip()]

def on_load_filters_clicked(_):
    aansluitnummers = get_aansluit_list()
    if not aansluitnummers:
        with output_area:
            clear_output()
            print("Geen geldige EANs ingevuld.")
        return
    with output_area:
        clear_output()
        print("Zoeken naar TypeIDs...")
    for num in aansluitnummers:
        tlist = fetch_typeids_for_aansluiting(num)
        with output_area:
            if tlist:
                print(f"{num}: {len(tlist)} TypeIDs gevonden.")
            else:
                print(f"{num}: geen TypeIDs gevonden (EAN mogelijk niet aanwezig).")
    btn_build_dataset.disabled = False

def on_reset_filters_clicked(_):
    global combined_data
    ean_textarea.value = "\n".join(prefilled_eans)
    start_date_picker.value = date.today() - relativedelta(years=2)
    end_date_picker.value = date.today()
    freq_selector.value = 'auto'
    btn_build_dataset.disabled = False
    btn_view_dataset.disabled = True
    btn_download_csv.disabled = True
    btn_download_excel.disabled = True
    combined_data = None
    with output_area:
        clear_output()
        print("Filters gereset.")

def on_build_dataset_clicked(_):
    global combined_data
    with output_area:
        clear_output()
        print("Data worden opgehaald...")

    progress_container.layout.visibility = 'visible'
    update_progress(5, "Start data ophalen")

    allowed, wait_time = check_rate_limit()
    if not allowed:
        with output_area:
            print(f"Rate limit overschreden. Wacht {wait_time} sec.")
        finish_progress()
        return

    aansluitnummers = get_aansluit_list()
    if not aansluitnummers:
        with output_area:
            print("Geen geldige EANs.")
        finish_progress()
        return

    if not start_date_picker.value or not end_date_picker.value:
        with output_area:
            print("Selecteer start/einddatum.")
        finish_progress()
        return

    sdate = datetime.combine(start_date_picker.value, datetime.min.time())
    edate = datetime.combine(end_date_picker.value, datetime.min.time())
    freq_val = freq_selector.value

    update_progress(10, f"{len(aansluitnummers)} EANs, data ophalen...")

    try:
        combined_data = build_multiean_data(
            aansluitnummers, sdate, edate, freq_val,
            progress_callback=update_progress
        )
    except Exception as e:
        logger.error(f"Fout bij data ophalen: {e}")
        update_progress(100, "Fout!", True)
        with output_area:
            print("Fout bij data ophalen.")
        finish_progress()
        return

    if combined_data is None or combined_data.empty:
        update_progress(100, "Geen data gevonden", True)
        with output_area:
            print("Geen data / ongeldige EANs.")
        finish_progress()
        return

    update_progress(95, "Afronden...")
    time.sleep(0.5)
    update_progress(100, "Data klaar")
    with output_area:
        print(f"Data: {len(aansluitnummers)} EAN(s), shape={combined_data.shape}")

    btn_view_dataset.disabled = False
    btn_download_csv.disabled = False
    btn_download_excel.disabled = False
    finish_progress()

def on_view_dataset_clicked(_):
    global combined_data
    data_table_output.layout.display = 'block'
    with data_table_output:
        clear_output(wait=True)
        if combined_data is None or combined_data.empty:
            print("Geen data.")
        else:
            df_show = combined_data.head(200)
            display(HTML(df_show.to_html(index=False)))

def on_download_csv_clicked(_):
    global combined_data
    if combined_data is None or combined_data.empty:
        with output_area:
            clear_output()
            print("[CSV] Geen data.")
        return

    with output_area:
        clear_output()

    progress_container.layout.visibility = 'visible'
    update_progress(20, "CSV-export...")
    downloads = os.path.join(os.path.expanduser("~"), "Downloads")
    if not os.path.isdir(downloads):
        downloads = os.getcwd()
    start_date_str = start_date_picker.value.strftime("%Y%m%d")
    end_date_str = end_date_picker.value.strftime("%Y%m%d")
    fname = os.path.join(downloads, f"Dataset_{start_date_str}_tot_{end_date_str}.csv")

    if export_dataset_to_csv(combined_data, fname):
        update_progress(80, "CSV OK...")
        time.sleep(0.3)
        update_progress(100, "Klaar")
        with output_area:
            print(f"CSV in: {fname}")
    else:
        update_progress(100, "CSV mislukt", True)
    finish_progress()

def on_download_excel_clicked(_):
    global combined_data
    if combined_data is None or combined_data.empty:
        with output_area:
            clear_output()
            print("[XLS] Geen data.")
        return

    with output_area:
        clear_output()

    progress_container.layout.visibility = 'visible'
    update_progress(20, "XLS-export...")
    downloads = os.path.join(os.path.expanduser("~"), "Downloads")
    if not os.path.isdir(downloads):
        downloads = os.getcwd()
    start_date_str = start_date_picker.value.strftime("%Y%m%d")
    end_date_str = end_date_picker.value.strftime("%Y%m%d")
    fname = os.path.join(downloads, f"Dataset_{start_date_str}_tot_{end_date_str}.xlsx")

    if export_dataset_to_excel(combined_data, fname):
        update_progress(80, "XLS bijna klaar...")
        time.sleep(0.3)
        update_progress(100, "XLS OK")
        with output_area:
            print(f"Excel: {fname}")
    else:
        update_progress(100, "XLS mislukt", True)
    finish_progress()

btn_load_filters.on_click(on_load_filters_clicked)
btn_reset_filters.on_click(on_reset_filters_clicked)
btn_build_dataset.on_click(on_build_dataset_clicked)
btn_view_dataset.on_click(on_view_dataset_clicked)
btn_download_csv.on_click(on_download_csv_clicked)
btn_download_excel.on_click(on_download_excel_clicked)